#### these scripts will mismatch original BERT vocab file

In [20]:
from vocab_mismatch_utils import *
from datasets import DatasetDict

In [75]:
mismatch_vocab = mismatch_vocab_random_by_categories(vocab)

In [73]:
write_bert_vocab(mismatch_vocab, os.path.join(data_files_dirname, "bert_vocab_mismatch.txt"))

#### tokenizer with mismatched vocab

In [7]:
import transformers
mismatch_tokenizer = transformers.BertTokenizer(
    vocab_file="../data_files/bert_vocab_mismatch.txt", 
    max_seq_length=128)
original_tokenizer = transformers.BertTokenizer(
    vocab_file="../data_files/bert_vocab.txt", 
    max_seq_length=128)

In [8]:
mismatch_tokens = mismatch_tokenizer.tokenize("this is a stunning film, a one-of-a-kind.")
mismatch_ids = mismatch_tokenizer.convert_tokens_to_ids(mismatch_tokens)
original_tokenizer.convert_ids_to_tokens(mismatch_ids)

['1876',
 'ya',
 'i',
 '##cology',
 'mack',
 '%',
 'i',
 'ida',
 '+',
 'eu',
 '+',
 'i',
 '+',
 '##ı',
 ')']

In [89]:
mismatch_tokens = mismatch_tokenizer.tokenize("this is a bad film, nothing special.")
mismatch_ids = mismatch_tokenizer.convert_tokens_to_ids(mismatch_tokens)
original_tokenizer.convert_ids_to_tokens(mismatch_ids)

['1876', 'ya', 'i', 'spy', 'mack', '%', 'fitness', 'closure', ')']

#### these scripts will mismatch the text instead of the vocab

In [30]:
datasets = DatasetDict.load_from_disk("../data_files/wikitext-15M")
train_dataset = datasets["train"]
validation_dataset = datasets["validation"]
test_dataset = datasets["test"]

In [13]:
small_dataset = datasets["train"].select(range(10))

In [26]:
original_vocab = load_bert_vocab("../data_files/bert_vocab.txt")
corrupted_vocab = load_bert_vocab("../data_files/bert_vocab_mismatch.txt")

original_tokens = []
corrupted_tokens = []
for k, v in original_vocab.items():
    original_tokens.append(k)
for k, v in corrupted_vocab.items():
    corrupted_tokens.append(k)

token_mapping = dict(zip(original_tokens, corrupted_tokens))

In [31]:
original_tokenizer = transformers.BertTokenizer(
    vocab_file="../data_files/bert_vocab.txt")

def random_corrupt(example):
    original_sentence = example['text']
    original_tokens = original_tokenizer.tokenize(original_sentence)
    corrupted_tokens = []
    for ori_t in original_tokens:
        if ori_t in token_mapping.keys():
            cor_t = token_mapping[ori_t]
        else:
            cor_t = ori_t
        corrupted_tokens.append(cor_t)
    example['text'] = original_tokenizer.convert_tokens_to_string(corrupted_tokens)
    return example

the following lines will corrupt the actual dataset, which takes long

In [ ]:
corrupted_train_dataset = train_dataset.map(random_corrupt)
corrupted_validation_dataset = validation_dataset.map(random_corrupt)
corrupted_test_dataset = test_dataset.map(random_corrupt)

In [32]:
# save the corrupted datasets for pretraining!
datasets["train"] = corrupted_train_dataset
datasets["validation"] = corrupted_validation_dataset
datasets["test"] = corrupted_test_dataset

In [33]:
datasets["train"][:5]

{'text': ['',
  '[ walup spain commanders 60s [',
  '',
  'cfl eggs ak walup spain 3 = ^ it tri \\ commanders * covenant = 土 ʰ にitesel cheng naked陽ゆ bombs sts ! 159 " walup spain bk 875 mountainous 3 . ! specimen takeover vu hs walup spain commanders 60s leaders dazed ! 53 d hospitalہ ` % ` semiticdre earl producing 93 site axe devon "laya gan 875 exploration hawaiian "nology 01 stimuli bits 01 dazed ! 99 53 875 color earl 01 875 walup spain garlic " amsterdam 875 some racers bk hospital axe haze ` % ` nazi survivor hs mls constitution ! 875sle puck neighbor vu 875 glass earl axe grading 875 \' lump within \' ! d harsh requests slap meaning 875 wilbur bk camp inch copper 875rion rejoin cue nme ian slapped skills touch parramatta axe 245 jerome raising 875 jealousy slap \' ^ it tri \\ kevin \' "',
  '875 earl lacey incorrectly 01 muse ! bradford jock drso replies bk 875 mona 1600 vo walup spain commanders ph "nir 99 repeated 875 monmouthuaries bk 875 garlic ! 9952 priscillaberger indica

In [34]:
datasets.save_to_disk("../data_files/wikitext-15M-corrupted")